In [1]:
import tensorflow

from cnn import CNN
from reader import Reader
from sklearn.metrics import classification_report, confusion_matrix 
import numpy as np
from tensorflow.keras.utils import to_categorical


In [2]:
EPOCHS = 10
BATCH_SIZE = 128
NUM_CLASSES = 2
LEARN_RATE = 0.01
EMBED_SIZE = 50
FILENAME = "data/twitter_data.pkl"
OVERSAMPLING_RATE = 3
VOCAB_LEN = 10000
MAX_LEN = 100

reader = Reader(filename=FILENAME, num_classes=NUM_CLASSES, vocab_len=VOCAB_LEN)
X, y = reader.load()

mapping = {'racism':1,'sexism':1,'none':0}
y = [mapping[b] for b in y]

# Oversampling before split
# hate = [i for i in range(len(y)) if y[i]==1]
# X = X + [X[x] for x in hate]*(OVERSAMPLING_RATE-1)
# y = y +  [1 for i in range(len(hate))]*(OVERSAMPLING_RATE-1)

X_train, X_test, y_train, y_test = reader.split(X, y)

# Oversampling after split
hate = [i for i in range(len(y_train)) if y_train[i]==1]
X_train = X_train + [X_train[x] for x in hate]*(OVERSAMPLING_RATE-1)
y_train = y_train + [1 for i in range(len(hate))]*(OVERSAMPLING_RATE-1)

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [3]:
model = CNN(max_len=MAX_LEN,
          num_classes=NUM_CLASSES, 
          batch_size=BATCH_SIZE, 
          epochs=EPOCHS, 
          embed_size=EMBED_SIZE, 
          vocab_len=VOCAB_LEN,
          save_model=True,
          checkpoint_path="results/cnn-10-epochs-hoseem-2-classes.ckpt", 
          save_path="results/cnn-10-epochs-hoseem-2-classes")

In [4]:
model.fit(X_train, y_train)

Epoch 1/10
185/185 [==============================] - ETA: 0s - loss: 0.9434 - accuracy: 0.6657
Epoch 1: saving model to results\cnn-10-epochs-hoseem-2-classes.ckpt
185/185 [==============================] - 38s 116ms/step - loss: 0.9434 - accuracy: 0.6657
Epoch 2/10
184/185 [============================>.] - ETA: 0s - loss: 0.4182 - accuracy: 0.8633
Epoch 2: saving model to results\cnn-10-epochs-hoseem-2-classes.ckpt
185/185 [==============================] - 10s 55ms/step - loss: 0.4181 - accuracy: 0.8634
Epoch 3/10
185/185 [==============================] - ETA: 0s - loss: 0.3017 - accuracy: 0.9156
Epoch 3: saving model to results\cnn-10-epochs-hoseem-2-classes.ckpt
185/185 [==============================] - 10s 56ms/step - loss: 0.3017 - accuracy: 0.9156
Epoch 4/10
184/185 [============================>.] - ETA: 0s - loss: 0.2342 - accuracy: 0.9408
Epoch 4: saving model to results\cnn-10-epochs-hoseem-2-classes.ckpt
185/185 [==============================] - 10s 54ms/step - loss: 0

INFO:tensorflow:Assets written to: results/cnn-10-epochs-hoseem-2-classes\assets


INFO:tensorflow:Assets written to: results/cnn-10-epochs-hoseem-2-classes\assets


In [5]:
predictions = model.predict(X_test)
classes = np.argmax(y_test, axis=1)

predictions = list(map(lambda x: int(x), predictions))
print(confusion_matrix(classes, predictions))
print(classification_report(classes, predictions))

51/51 [==============================] - 4s 53ms/step
[[979 125]
 [124 381]]
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      1104
           1       0.75      0.75      0.75       505

    accuracy                           0.85      1609
   macro avg       0.82      0.82      0.82      1609
weighted avg       0.85      0.85      0.85      1609



In [7]:
model = CNN.load("results/cnn-10-epochs-hoseem-2-classes")
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis=1)
classes = np.argmax(y_test, axis=1)

predictions = list(map(lambda x: int(x), predictions))
print(confusion_matrix(classes, predictions))
print(classification_report(classes, predictions))

51/51 [==============================] - 1s 17ms/step
[[979 125]
 [124 381]]
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      1104
           1       0.75      0.75      0.75       505

    accuracy                           0.85      1609
   macro avg       0.82      0.82      0.82      1609
weighted avg       0.85      0.85      0.85      1609

